In [1]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Ensure all rows and columns are printed
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

In [ ]:
# load in the mouse and human atlases

human_labels = sitk.ReadImage("./human/resampled_segmentation_mask.nii")
mouse_labels = sitk.ReadImage("./Mouse/BorglundData/labelsReformat.nii")

human_labels_array = sitk.GetArrayFromImage(human_labels)
mouse_labels_array = sitk.GetArrayFromImage(mouse_labels)

print("human_labels_array dim:", human_labels_array.shape)
print("mouse_labels_array dim:", mouse_labels_array.shape)

human_labels_df = pd.read_csv("./human/CerebrA_LabelDetails.csv")
mouse_labels_df = pd.read_csv("./Mouse/BorglundData/c57_brain_atlas_labels.csv")

# to aid with string matching convert all labels to lowercase
human_labels_df["Label Name"] = human_labels_df["Label Name"].astype(str).str.lower()

# Other overlapping labels include:

common_label_diff_names = { # "label_mouse": "label_human"
    "optic tract": "optic chiasm",
    "arbor vita of cerebellum": "cerebellum white matter",
    "nucleus accumbens": "accumbens area",
    "globus pallidus": "pallidum",
}


In [ ]:
# relabel the regions with the same values in the mouse and human datasets

# Find common elements
common = list(set(human_labels_df["Label Name"]) & set(mouse_labels_df["Structure"]))

# Print results
print("Number of common labels:", len(common))
print("Common labels:", common)



# want to give these the same labels to display in seg masks!!
# just relabel them to be greater the max of both images
# and relabel the intensity in the image
max_int = int(np.maximum(np.max(human_labels_array), np.max(mouse_labels_array)))

# check that matches the max label in dfs
max_dfs = np.maximum(np.max(human_labels_df[["RH Label", "LH Labels"]]), np.max(mouse_labels_df[["right label", "left label"]]))

new_label_val = max_int + 1

## ISSUE: woith hjow the keys are being extracted
dict_keys = [i for i in common_label_diff_names.keys()]
common += dict_keys
print("Updated common labels:",common)


# now relabel starting at 251, 252 pair
for label in common:
    print("label:", label)
    if (label in common_label_diff_names):
        label_mouse = label
        label_human = common_label_diff_names[label]
    else:
        label_mouse = label
        label_human = label

    # change the intensities in the image
    # get current intesities to know what pxls to filter
    human_right_intensity = human_labels_df["RH Label"][human_labels_df["Label Name"] == label_human].values[0]
    human_left_intensity = human_labels_df["LH Labels"][human_labels_df["Label Name"] == label_human].values[0]

    mouse_right_intensity = mouse_labels_df["right label"][mouse_labels_df["Structure"] == label_mouse].values[0]
    mouse_left_intensity = mouse_labels_df["left label"][mouse_labels_df["Structure"] == label_mouse].values[0]

    # overwrite these intensity values with the new ones
    # overwritting values in a numpy array
    human_labels_array[human_labels_array == human_right_intensity] = new_label_val
    mouse_labels_array[mouse_labels_array == mouse_right_intensity] = new_label_val

    human_labels_array[human_labels_array == human_left_intensity] = new_label_val + 1
    mouse_labels_array[mouse_labels_array == mouse_left_intensity] = new_label_val + 1

    
    # relabel both df labels
    human_labels_df.loc[human_labels_df["Label Name"] == label_human, "RH Label"] = new_label_val
    mouse_labels_df.loc[mouse_labels_df["Structure"] == label_mouse, "right label"] = new_label_val
    new_label_val += 1

    human_labels_df.loc[human_labels_df["Label Name"] == label_human, "LH Labels"] = new_label_val
    mouse_labels_df.loc[mouse_labels_df["Structure"] == label_mouse, "left label"] = new_label_val

    new_label_val += 1



In [ ]:
# save the updated labeled atlases
mouse_labels_img = sitk.GetImageFromArray(mouse_labels_array)
human_labels_img = sitk.GetImageFromArray(human_labels_array)

# resample the smaller mask to the larger one
reference_image = mouse_labels_img # since larger
moving_image = human_labels_img # since smaller

# linear transform??
resampled_image = sitk.Resample(moving_image, reference_image)

human_labels_img = resampled_image

# Save result


sitk.WriteImage(mouse_labels_img, "mouse_labels_common.nii")
sitk.WriteImage(human_labels_img, "human_labels_common.nii")

mouse_labels_df.to_csv("mouse_labels_common.csv", index=False)
human_labels_df.to_csv("human_labels_common.csv", index=False)



In [ ]:
human_labels_df

In [ ]:
mouse_labels_df